In [1]:
import sys
sys.path.append("slim")
from picsellia import Client
import main
from util.infer import infer
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
token = "f2a5daec-e0cc-4ea8-a5eb-10d04fd1e153" # Token from the picsell-IA platform
model_picked = "faster_rcnn" # Choose your base model here from the list of possible models
model_name = "faster_rcnn" # Name your to-be trained model
annotation_type = "rectangle" # Chose the type of annotation used

batch_size = 1
learning_rate = None #You can let this value to None
nb_steps = 20000
mask_type = None #Set this to 'PNG_MASKS' if you want to train a mask segmentation model.

In [ ]:
def train(model_picked, model_name, annotation_type, batch_size, 
     learning_rate, nb_steps, mask_type, min_score_tresh=0.8):
    
    path_models = "models/"
    model_selected = path_models + model_picked + "/"

    clt = Client(token=token, host="https://backstage.picsellia.com/sdk/")
    clt.init_model(model_name)

    clt.dl_annotations()
    clt.generate_labelmap()
    clt.local_pic_save()

    main.create_record_files(label_path=clt.label_path, record_dir=clt.record_dir, 
                             tfExample_generator=clt.tf_vars_generator, annotation_type=annotation_type)
    
    if clt.training_id!=0:
        previous_path = clt.base_dir.split("/")[:-1]
        previous_path[-1] = clt.training_id - 1
        model_selected = "{}/{}/{}/".format(*previous_path)+"checkpoint/"
    
    main.edit_config(model_selected=model_selected, config_output_dir=clt.config_dir,
                record_dir=clt.record_dir, 
                label_map_path=clt.label_path, 
                masks=mask_type, 
                num_steps=nb_steps,
                batch_size=batch_size, 
                learning_rate=learning_rate,
                training_id=clt.training_id)
    
    main.legacy_train(ckpt_dir=clt.checkpoint_dir, 
                     conf_dir=clt.config_dir)
    
    dict_log = main.tfevents_to_dict(path=clt.checkpoint_dir)
    clt.send_logs(dict_log)
    tf.reset_default_graph()
    
    main.export_infer_graph(ckpt_dir=clt.checkpoint_dir, 
                       exported_model_dir=clt.exported_model, 
                       pipeline_config_path=clt.config_dir,
                       write_inference_graph=True, input_type="image_tensor", input_shape=None)
    
    infer(clt.eval_list, exported_model_dir=clt.exported_model, 
              label_map_path=clt.label_path, results_dir=clt.results_dir, min_score_thresh=min_score_thresh)
    clt.send_examples()
    

In [ ]:
train(model_picked=model_picked, model_name=model_name, annotation_type=annotation_type, batch_size=batch_size, 
     learning_rate=learning_rate, nb_steps=nb_steps, mask_type=mask_type)